<a href="https://colab.research.google.com/github/heartsker/TicTacToeRL/blob/main/TicTacToeRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tic Tac Toe with Reinforcement learning

## Game

### Rules
Tic tac toe is played on the 3-by-3 field and it involves 2 players.
First one is playing for crosses and second - for zeroes.
The aim is to get three in a row.

## Reinforcement learning

### Definition
Reinforcement learning is an area of Machine Learning. It is about taking suitable action to maximize reward in a particular situation. It is employed by various software and machines to find the best possible behavior or path it should take in a specific situation. Reinforcement learning differs from supervised learning in a way that in supervised learning the training data has the answer key with it so the model is trained with the correct answer itself whereas in reinforcement learning, there is no answer but the reinforcement agent decides what to do to perform the given task. In the absence of a training dataset, it is bound to learn from its experience. 

### Main points in Reinforcement learning
- **Input:** The input should be an initial state from which the model will start
- **Output:** There are many possible outputs as there are a variety of solutions to a particular problem
- **Training:** The training is based upon the input, The model will return a state and the user will decide to reward or punish the model based on its output.
- The model keeps continues to learn.
- The best solution is decided based on the maximum reward.

### Types of Reinforcement
There are two types of Reinforcement: 
- Positive

    Positive Reinforcement is defined as when an event, occurs due to a particular behavior, increases the strength and the frequency of the behavior. In other words, it has a positive effect on behavior.
    Advantages of **positive** reinforcement learning are: 
    - Maximizes Performance
    - Sustain change for a long period of time
    - Too much reinforcement can lead to an overload of states which can diminish the results
    
- Negative

    Negative Reinforcement is defined as strengthening of behavior because a negative condition is stopped or avoided. 
    Advantages of **negative** reinforcement learning: 

    - Increases Behavior
    - Provide defiance to a minimum standard of performance
    - It Only provides enough to meet up the minimum behavior

### Various Practical applications of Reinforcement Learning (RL)
 
RL can be used
- in robotics for industrial automation
- in machine learning and data processing
- to create training systems that provide custom instruction and materials according to the requirement of students
- in large environments in the following situations: 

### Appropriate situations to use RL

RL can be used in large environments in the following situations: 
- A model of the environment is known, but an analytic solution is not available
- Only a simulation model of the environment is given *(the subject of simulation-based optimization)*
- The only way to collect information about the environment is to interact with it

# The game invironment

First, we need to define some global parameters for our game.

In [ ]:
boardSize = 3

Now, let's create our game invironment.

That will be 3-by-3 board filled with zeroes.

In [ ]:
def getNewBoard():
    return [[0 for i in range(boardSize)] for i in range(boardSize)]

print(getNewBoard())

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]


Also let us create a function to get **valid** moves:

In [ ]:
def getValidMoves(board):
    moves = []
    for i in range(boardSize):
        for j in range(boardSize):
            if board[i][j] == 0:
                moves.append((i, j))
    return moves

It's time to add function to check whether some player has won:

In [ ]:
# returns 1 or 2 if player 1 or 2 wins
# returns 0 if that is a draw
# returns -1 if game should continue
def getWinner(board):
    winner = 0

    # check rows
    for i in range(boardSize):
        player = board[i][0]
        win = True
        if player == 0:
            continue
        for j in range(boardSize):
            if board[i][j] != player:
                win = False
                break
        if win:
            return player

    # check columns
    for i in range(boardSize):
        player = board[0][i]
        win = True
        if player == 0:
            continue
        for j in range(boardSize):
            if board[j][i] != player:
                win = False
                break
        if win:
            return player

    # check diagonal (\)
    player = board[0][0]
    if player != 0:
        win = True
        for i in range(boardSize):
            if board[i][i] != player:
                win = False
                break
        if win:
            return player

    # check diagonal (/)
    player = board[boardSize - 1][0]
    if player != 0:
        win = True
        for i in range(boardSize):
            if board[boardSize - i - 1][i] != player:
                win = False
                break
        if win:
            return player

    # check if game is other
    if (len(getValidMoves(board)) == 0):
        # that is draw
        return 0
    else:
        # there are moves to play
        return -1

Finally, we need a function for a nice looking visualization of our game:

In [ ]:
def visualize(board):
    print('/-----\\')
    for i in range(boardSize):
        for j in range(boardSize):
            print('|', end = '')
            print('X' if board[i][j] == 1 else 'O' if board[i][j] == 2 else ' ', end = '')
        print('|')
        if i != boardSize - 1:
            print('-------')
    print('\\-----/')

# The game simulator

Now that we’ve created our tic-tac-toe environment, we need a way to simulate actual games. While there are several ways we can do this, we’re going to use a random simulator. There a couple reasons why this is a good idea.

First, a random simulator will create the broadest possible collection of training data. It means when a human plays against our AI-trained model, the AI won’t be easily flummoxed by unintuitive moves that it wasn’t trained on. In a more complicated game, we might need some sort of heuristics to narrow the search space and point the training simulations in the general right direction, but in tic-tac-toe the search space is small so this isn’t a problem.


Second, a random simulator will give us a good baseline for measuring the performance of our model against. In tic-tac-toe, ‘X’ has an inherent edge because it goes first, but we don’t know just how much of an advantage this gives ‘X’ without simulating it.

In [ ]:
import random

In [ ]:
def getBestMove(board, model, player, rnd=0):
    scores = []
    moves = getValidMoves(board)
    
    # make predictions for each possible move
    for i in range(len(moves)):
        future = np.array(board)
        future[moves[i][0]][moves[i][1]] = player
        prediction = model.predict(future.reshape((-1, 9)))[0]
        if player == 1:
            winPrediction = prediction[1]
            lossPrediction = prediction[2]
        else:
            winPrediction = prediction[2]
            lossPrediction = prediction[1]
        drawPrediction = prediction[0]
        if winPrediction - lossPrediction > 0:
            scores.append(winPrediction - lossPrediction)
        else:
            scores.append(drawPrediction - lossPrediction)

    # choose the best move with a random factor
    bestMoves = np.flip(np.argsort(scores))
    for i in range(len(bestMoves)):
        if random.random() * rnd < 0.5:
            return moves[bestMoves[i]]

    # choose a move completely at random
    return random.choice(moves)

In [ ]:
def makeMove(board, player, move):
    board[move[0]][move[1]] = player
    return board

In [ ]:
def simulateGame(p1=None, p2=None, rnd=0):
    history = []
    board = getNewBoard()
    playerToMove = 1
    
    while getWinner(board) == -1:
        # chose a move (random or use a player model if provided)
        move = None
        if playerToMove == 1 and p1 != None:
            move = getBestMove(board, p1, playerToMove, rnd)
        elif playerToMove == 2 and p2 != None:
            move = getBestMove(board, p2, playerToMove, rnd)
        else:
            move = random.choice(getValidMoves(board))
        
        # make the move
        board = makeMove(board, playerToMove, move)
        
        # add the move to the history
        history.append((playerToMove, move))
        
        # switch the active player
        playerToMove = playerToMove % 2 + 1
        
    return history

We see that the “history” of a game consists of an array of tuples.

The first element in each of these tuples is the player who moved (1 for ‘X’, 2 for ‘O’), and the second element is the coordinate where that player moved.

Any board state from this game can be easily reconstructed from this array. Let’s create a function to do exactly that.

In [ ]:
def restoreBoard(moves):
    board = getNewBoard()
    for move in moves:
        board = makeMove(board, move[0], move[1])
    return board

Now, we need a function to generate statistics from this list of games.

In [ ]:
def gameSeriesInfo(games, player=1):
    win = 0
    loss = 0
    draw = 0
    for game in games:
        result = getWinner(restoreBoard(game))
        if result == -1:
            continue
        elif result == player:
            win += 1
        elif result == 0:
            draw += 1
        else:
            loss += 1
    
    winPct = win / len(games) * 100
    lossPct = loss / len(games) * 100
    drawPct = draw / len(games) * 100

    print("Results for player %d:" % (player))
    print("Wins: %d (%.1f%%)" % (win, winPct))
    print("Loss: %d (%.1f%%)" % (loss, lossPct))
    print("Draw: %d (%.1f%%)" % (draw, drawPct))

# Simulating random games

Now that we’re at a point where we can simulate random games, let’s generate some aggregate statistics for 10,000 games at once. We can do this in a single line.

In [ ]:
games = [simulateGame() for _ in range(10000)]

So, how did they go?

In [ ]:
gameSeriesInfo(games)

Results for player 1:
Wins: 5874 (58.7%)
Loss: 2873 (28.7%)
Draw: 1253 (12.5%)


Not very impressing, huh?

By the way, we see that player 1 (‘X’) wins about half of the time, while player 2 (‘O’) only wins about a quarter of the time. The players also draw about a quarter of the time. This is the baseline to beat for our model.

# The DNN Model

It’s now time to build a deep neural network! Remember that our **input** data is individual board states and our **output** data used for classification is the resulting outcome for each game associated with that board state.


The **input** data is thus a nine-element array, if we flatten the board state. The **output** data is a three-element array, if we one-hot encode the possible outcomes of a game (‘X’ wins, ‘O’ wins, draw).

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.backend import reshape
from keras.utils.np_utils import to_categorical

def getModel():
    numCells = 9
    outcomes = 3
    model = Sequential()
    model.add(Dense(200, activation='relu', input_shape=(9, )))
    model.add(Dropout(0.2))
    model.add(Dense(125, activation='relu'))
    model.add(Dense(75, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(outcomes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

Note how the loss function is categorical crossentropy.
This is how we tell the model that we want it to output an array of probabilities.

These probabilities project the model’s confidence in each of the three game outcomes for a given board.
Remember the random games we simulated? These are our training data. But we need to format the data in such a way that our model can consume it.

This means doing the following:
- Splitting each game into separate board states, labelling each board with the eventual outcome of its associated game
- Flattening each board into a 1D array

We also want to reserve a test dataset for validation purposes. Here’s the function we use to do all of the above.

In [ ]:
import numpy as np

def gamesToWinLossData(games):
    X = []
    y = []
    for game in games:
        winner = getWinner(restoreBoard(game))
        for move in range(len(game)):
            X.append(restoreBoard(game[:(move + 1)]))
            y.append(winner)

    X = np.array(X).reshape((-1, 9))
    y = to_categorical(y)
    
    # return an appropriate train/test split
    trainNum = int(len(X) * 0.8)
    return (X[:trainNum], X[trainNum:], y[:trainNum], y[trainNum:])

Now, we can instantiate our model, reshape our data, and train the model.

It will take a while...

In [ ]:
model = getModel()
X_train, X_test, y_train, y_test = gamesToWinLossData(games)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100)

Epoch 1/100
611/611 [==============================] - 3s 4ms/step - loss: 0.9341 - acc: 0.5768 - val_loss: 0.8917 - val_acc: 0.5973
Epoch 2/100
611/611 [==============================] - 2s 3ms/step - loss: 0.8802 - acc: 0.6004 - val_loss: 0.8392 - val_acc: 0.6218
Epoch 3/100
611/611 [==============================] - 2s 3ms/step - loss: 0.8423 - acc: 0.6132 - val_loss: 0.8053 - val_acc: 0.6286
Epoch 4/100
611/611 [==============================] - 2s 3ms/step - loss: 0.8152 - acc: 0.6248 - val_loss: 0.7862 - val_acc: 0.6364
Epoch 5/100
611/611 [==============================] - 2s 3ms/step - loss: 0.7965 - acc: 0.6342 - val_loss: 0.7720 - val_acc: 0.6418
Epoch 6/100
611/611 [==============================] - 2s 3ms/step - loss: 0.7841 - acc: 0.6389 - val_loss: 0.7609 - val_acc: 0.6493
Epoch 7/100
611/611 [==============================] - 2s 3ms/step - loss: 0.7727 - acc: 0.6468 - val_loss: 0.7524 - val_acc: 0.6544
Epoch 8/100
611/611 [==============================] - 2s 3ms/step - 

# Model perfomance

Once the training is done, we can see how well our model performs!

Let’s pass our newly trained model to the simulator as ‘X’ and have it play against a random ‘O’ for 1,000 games.

This takes a few seconds, but when it’s done, we can use the same statistics function as before to see what the outcome was.

In [ ]:
gamesModelRandom = [simulateGame(p1=model) for _ in range(1000)]
gameSeriesInfo(gamesModelRandom)

Results for player 1:
Wins: 977 (97.7%)
Loss: 6 (0.6%)
Draw: 17 (1.7%)


Wow! We’ve gone from losing 24.2% of games to only losing 1.6% of the time!

Let’s see how our model fares as ‘O’, playing against a random ‘X’.

In [ ]:
gamesRandomModel = [simulateGame(p2=model) for _ in range(1000)]
gameSeriesInfo(gamesRandomModel, player=2)

Results for player 2:
Wins: 315 (31.5%)
Loss: 50 (5.0%)
Draw: 635 (63.5%)


This is equally impressive! ‘O’ has gone from losing 51.9% of games to only losing 4.3% of the time.

What happens when the model is used to play both sides? To avoid complete determinism, we apply a random factor to force the model to choose different moves each game.

In [ ]:
gamesModelModel = [simulateGame(p1=model, p2=model, rnd=0.6) for _ in range(1000)]
gameSeriesInfo(gamesModelModel, player=1)
print()
gameSeriesInfo(gamesModelModel, player=2)

Results for player 1:
Wins: 562 (56.2%)
Loss: 59 (5.9%)
Draw: 379 (37.9%)

Results for player 2:
Wins: 59 (5.9%)
Loss: 562 (56.2%)
Draw: 379 (37.9%)


We see that the game tends toward a draw, which is what we would expect from a couple of human players.

Still, neither player is particularly good. Remember that in perfect play, games *always* end in a draw.

As a final measure of skill, let’s see how the length of the average game has changed.

We would expect this to go down with an increased imbalance in skill levels.

In [ ]:
print("Average length of fully random game is %f moves" % (np.mean([float(len(game)) for game in games])))
print("Average length of game where P1 uses NN is %f moves" % (np.mean([float(len(game)) for game in gamesModelRandom])))
print("Average length of game where P2 uses NN is %f moves" % (np.mean([float(len(game)) for game in gamesRandomModel])))
print("Average length of game where both use NN is %f moves" % (np.mean([float(len(game)) for game in gamesModelModel])))

Average length of fully random game is 7.826800 moves
Average length of game where P1 uses NN is 6.807000 moves
Average length of game where P2 uses NN is 8.023000 moves
Average length of game where both use NN is 7.777000 moves


As shown above, the games are a move shorter for ‘X’ and about the same for ‘O’.

When the model is playing both sides, the games actually tend to be slightly longer than when they’re fully random.

# ML vs. Me

Now, we play a game against our model and see how well it does. We let the model play as ‘X’.

In [ ]:
board = getNewBoard()
move = getBestMove(board, model, 1)
board = makeMove(board, 1, move)
visualize(board)

/-----\
| | | |
-------
| |X| |
-------
| | | |
\-----/


In [ ]:
board = makeMove(board, 2, [2, 2])
visualize(board)

/-----\
| | | |
-------
| |X| |
-------
| | |O|
\-----/


In [ ]:
move = getBestMove(board, model, 1)
board = makeMove(board, 1, move)
visualize(board)

/-----\
| | | |
-------
| |X| |
-------
| |X|O|
\-----/


In [ ]:
board = makeMove(board, 2, [0, 2])
visualize(board)

/-----\
| | |O|
-------
| |X| |
-------
| |X|O|
\-----/


In [ ]:
move = getBestMove(board, model, 1)
board = makeMove(board, 1, move)
visualize(board)

/-----\
| | |O|
-------
| |X|X|
-------
| |X|O|
\-----/


In [ ]:
board = makeMove(board, 2, [0, 1])
visualize(board)

/-----\
| |O|O|
-------
| |X|X|
-------
| |X|O|
\-----/


In [ ]:
move = getBestMove(board, model, 1)
board = makeMove(board, 1, move)
visualize(board)

/-----\
| |O|O|
-------
|X|X|X|
-------
| |X|O|
\-----/
